In [1]:
import pandas as pd
from PySimpleGUI import PySimpleGUI as sg
from tkinter import filedialog
import os
from ipykernel import kernelapp as app

# Criar Janela
sg.theme('Reddit')
layout = [ 
    [sg.Button('OK',button_color=('blue', 'white')),sg.Button('Cancel',button_color=('red', 'white')),sg.Button('Arquivo', target='Arquivo'), sg.Text('...',size=(40,1), key='...')]
]
janela = sg.Window('Escolha o arquivo', layout)
#text_element = janela['cep']
# Ler eventos
while True:
        eventos, valores = janela.read()
        if eventos == sg.WINDOW_CLOSED:
            break
        if eventos == 'Cancel':
            janela.close()
            break
        if eventos == 'Arquivo':
            arq = filedialog.askopenfilename()
            abs_arq = os.path.abspath(arq)
            janela['...'].update(abs_arq)
        if eventos == 'OK': 
            janela.close()
data = open(abs_arq).readlines()
#print(data)
#-----------------machines ----------------------------#
machines = int(data[0])
# -----------------jobs ------------------------------#
jobs = int(data[1])
 # --------------- times -------------------------------#
# segunda linha em diante até linha 2 + quantidade de jobs
str_times = ' '.join(str(i) for i in data[2: 4 + jobs]) # x[start:end:step]...x[:] means same as x[0:len(x):1]
times = [int(i) for i in str_times.split() if i.isdigit()]
print ('Máquinas:',machines)
print ("Atividades:",jobs)
print ("Tempos:",times)
# ------------------ setup --------------------------#
# depois da 2 + n jobs + machines (zeros) até  total de linhas - (2 + n jobs + machines) tempos de setup
jm = jobs + machines  # tarefas + maquinas dummys (linhas zeros)
str_setups = ' '.join(str(i) for i in data[2 + jm: len(data) - 2 + jm])
setups_list = [int(i) for i in str_setups.split() if i.isdigit()]
setups_matrix = [setups_list[i::jm] for i in range(jm)]
# matriz transposta para ficar igual ao apresentado na matriz
matriz_tempos = (pd.DataFrame(setups_matrix).T)
print ('Matriz transposta')
print (matriz_tempos)

Máquinas: 2
Atividades: 10
Tempos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0]
Matriz transposta
     0    1    2    3    4    5    6    7    8    9    10   11
0   500   93   48   22   65   91   16   81   48   87   20   20
1    56  500   94   93   96   48   81   60   49   27   87   87
2    55   49  500   82   54   21   68   20   25   20   59   59
3    17   21    6  500   83   87   14   82   42   76   42   42
4    29   34   37   71  500   66   37   66   28   52   52   52
5    28   35   38   72   12  500   89   10   46   29   74   74
6    29   50   46   80   28   64  500   56   28   66   11   11
7    69   63   73  103   49   95   94  500   54   90   43   43
8    26   30   17   26   41    8   16   31  500   19   68   68
9    51   31   33   19   54   30   50   12   21  500   89   89
10   32   57   43   58   52   40   19   66   47   93  500  500
11   32   57   43   58   52   40   19   66   47   93  500  500


In [2]:
# soma com os tempos
t = times
M1 = [[0 for j in range(jm)] for i in range(len(t))]
M1[0][0] = matriz_tempos[0][0]
for i in range(len(t)):
    for j in range(jm):
        if i == j:
            M1[i][j] = matriz_tempos[j][i] + 0
        else:
            M1[i][j] = matriz_tempos[j][i] + int(t[j])


print (pd.DataFrame(M1))
print (t)

     0    1    2    3    4    5    6    7    8    9    10   11
0   500   95   51   26   70   97   23   89   57   97   20   20
1    57  500   97   97  101   54   88   68   58   37   87   87
2    56   51  500   86   59   27   75   28   34   30   59   59
3    18   23    9  500   88   93   21   90   51   86   42   42
4    30   36   40   75  500   72   44   74   37   62   52   52
5    29   37   41   76   17  500   96   18   55   39   74   74
6    30   52   49   84   33   70  500   64   37   76   11   11
7    70   65   76  107   54  101  101  500   63  100   43   43
8    27   32   20   30   46   14   23   39  500   29   68   68
9    52   33   36   23   59   36   57   20   30  500   89   89
10   33   59   46   62   57   46   26   74   56  103  500  500
11   33   59   46   62   57   46   26   74   56  103  500  500
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0]


In [3]:
import numpy as np 
from scipy import sparse
# matriz dos tempos
a = np.array(M1)

# localizar a linha 11 e localizar o valor mínimo
m1 = [np.amin(a[10,:])]
w = np.argwhere(a[10,:] == m1)
# linha a ser iniciada a varredura e adicionada ao ibm
x = 10
ibm = []
ibm.append(x)
# Atribuir um valor elevado para não ser considerado
a[10,:] = 1000
a[:,w] = 1000

# localizar a linha 12 e localizar o valor mínimo
m2 = [np.min(a[11])]
# index
w = np.argwhere(a[11] == m2)
# linha a ser iniciada a varredura e adicionada ao icm
x = 11
icm = []
icm.append(x)
# Atribuir um valor elevado para não ser considerado
a[11,:] = 1000
a[:,w] = 1000
for columns in range(12):
        if len(m2) != 6: 
            print ("m1=",np.sum(m1))
            print ("m2=",np.sum(m2))
           # escolhe a máquina de menos demanda
            if np.sum(m1)<=np.sum(m2):
                b = np.amin(a) # menor elemento de toda matriz
                # localiza na matriz a o valor b
                w = np.argwhere(a == b)
                # separa em x e y
                x = w[0][0]
                y = w[0][1]
                # Atribuir um valor elevado para não ser considerado
                a[:,y] = 1000
                a[x,:] = 1000     
                ibm.append(x)
                m1.append(b)
                print ("m1 if",m1)
            else:
                c = np.amin(a) # menor elemento de toda matriz
                # localiza na matriz a o valor c
                w = np.argwhere(a == c)
                # separa em x e y
                x = w[0][0]
                y = w[0][1]
                # Atribuir um valor elevado para não ser considerado
                a[:,y] = 1000
                a[x,:] = 1000
                icm.append(x)
                m2.append(c)
                print ("m2 else",m2)
        else:             #________________________________________________
            break 
               
        #________________________________________________
          
            #s = np.sum(m1)
            #r = np.sum(m2)
            #print (s, r)
            #print (np.sum(m1), np.sum(m2))
            
print ('Solução encontrada a partir de Método Construtivo')
print ()
print ('Valores na máquina 1:',m1)
print ('---------------------------')
print ('Função Objetivo na máquina 1:', np.sum(m1))
print ('---------------------------')
print ('Atividades alocadas na máquina 1:',ibm)
print ('---------------------------')
print ('Valores na máquina 2:',m2)
print ('---------------------------')
print ('Função Objetivo na máquina 1:', np.sum(m2))
print ('---------------------------')
print ('Atividades alocadas na máquina 2:',icm)

m1= 26
m2= 33
m1 if [26, 9]
m1= 35
m2= 33
m2 else [33, 11]
m1= 35
m2= 44
m1 if [26, 9, 14]
m1= 49
m2= 44
m2 else [33, 11, 17]
m1= 49
m2= 61
m1 if [26, 9, 14, 20]
m1= 69
m2= 61
m2 else [33, 11, 17, 20]
m1= 69
m2= 81
m1 if [26, 9, 14, 20, 30]
m1= 99
m2= 81
m2 else [33, 11, 17, 20, 36]
m1= 99
m2= 117
m1 if [26, 9, 14, 20, 30, 58]
m1= 157
m2= 117
m2 else [33, 11, 17, 20, 36, 107]
Solução encontrada a partir de Método Construtivo

Valores na máquina 1: [26, 9, 14, 20, 30, 58]
---------------------------
Função Objetivo na máquina 1: 157
---------------------------
Atividades alocadas na máquina 1: [10, 3, 8, 0, 2, 1]
---------------------------
Valores na máquina 2: [33, 11, 17, 20, 36, 107]
---------------------------
Função Objetivo na máquina 1: 224
---------------------------
Atividades alocadas na máquina 2: [11, 6, 5, 9, 4, 7]
